In [1]:
# %pip install cdsapi

In [6]:
import os

api_key = "189586:2c9a82f3-46ff-4e11-aae6-6a8bcbe1bee5"
config_content = f"url: https://cds.climate.copernicus.eu/api/v2\nkey: {api_key}\n"

# Determine the home directory path
home_dir = os.path.expanduser("~")

# Create the .cdsapirc file with the configuration content
with open(os.path.join(home_dir, ".cdsapirc"), "w") as config_file:
    config_file.write(config_content)

In [12]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    "reanalysis-era5-land",
    {
        "variable": ["total_precipitation", "surface_solar_radiation_downwards"],
        "year": "2022",
        "month": [
            "06", "07", "08", "09", "10", "11", "12"
        ],
        "day": [
            "01", "02", "03", "04", "05", "06", "07", "08", "09", "10",
            "11", "12", "13", "14", "15", "16", "17", "18", "19", "20",
            "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"
        ],
        "time": [
            "00:00", "01:00", "02:00", "03:00", "04:00", "05:00",
            "06:00", "07:00", "08:00", "09:00", "10:00", "11:00",
            "12:00", "13:00", "14:00", "15:00", "16:00", "17:00",
            "18:00", "19:00", "20:00", "21:00", "22:00", "23:00"
        ],
        "area": [51.53, -0.013, 51.53, -0.013],
        "format": "netcdf",
    },
    "london_precipitation_solar_radiation_2022.nc",
)

2023-04-06 12:58:46,387 INFO Welcome to the CDS
2023-04-06 12:58:46,388 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-04-06 12:58:46,496 INFO Request is queued
2023-04-06 12:58:47,554 INFO Request is running
2023-04-06 13:13:06,953 INFO Request is completed
2023-04-06 13:13:06,954 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data5/adaptor.mars.internal-1680786692.9199011-5602-19-cfae5403-717b-456c-b6ec-a9d8f6442805.nc to london_precipitation_2022.nc (41.3K)
2023-04-06 13:13:07,478 INFO Download rate 78.9K/s


Result(content_length=42272,content_type=application/x-netcdf,location=https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data5/adaptor.mars.internal-1680786692.9199011-5602-19-cfae5403-717b-456c-b6ec-a9d8f6442805.nc)

In [9]:
# %pip install xarray netcdf4

     |████████████████████████████████| 973 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 5.2 MB 80.9 MB/s eta 0:00:01
     |████████████████████████████████| 12.3 MB 71.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 17.2 MB/s eta 0:00:01
ERROR: pandas-ml 0.6.1 requires enum34, which is not installed.
ERROR: fbprophet 0.7.1 requires cmdstanpy==0.9.5, which is not installed.
ERROR: responsibleai 0.23.0 has requirement ipykernel<=6.6.0, but you'll have ipykernel 6.8.0 which is incompatible.
ERROR: responsibleai 0.23.0 has requirement numba<0.54.0, but you'll have numba 0.55.2 which is incompatible.
ERROR: raiwidgets 0.23.0 has requirement itsdangerous==2.0.1, but you'll have itsdangerous 2.1.2 which is incompatible.
ERROR: datasets 2.3.2 has requirement dill<0.3.6, but you'll have dill 0.3.6 which is incompatible.
ERROR: dask-sql 2022.12.0 has requirement dask[dataframe,distributed]<=2022.11.1,>=2022.3.0, but you'll have dask 2021.6.0 which is incompa

In [13]:
import xarray as xr
import pandas as pd

# Load the NetCDF file using xarray
dataset = xr.open_dataset("london_precipitation_2022.nc")

# Convert the xarray Dataset to a pandas DataFrame
df = dataset.to_dataframe().reset_index()

# Drop the latitude and longitude columns, as they are constant
df = df.drop(columns=["latitude", "longitude"])

# Rename the columns for better readability
df = df.rename(columns={"time": "date", "tp": "precipitation"})

print(df.head())

   expver                date  precipitation
0       1 2022-06-01 00:00:00       0.002379
1       1 2022-06-01 01:00:00       0.000000
2       1 2022-06-01 02:00:00       0.000000
3       1 2022-06-01 03:00:00       0.000000
4       1 2022-06-01 04:00:00       0.000000


In [24]:
len(df.dropna(subset=["precipitation"]))

5136

In [28]:
df = df.dropna(subset=["precipitation"]).drop(columns = ['expver'])

In [29]:
df.to_csv("../data/precipitation_data_london_jun_dec.csv", sep=',', header = True)

In [30]:
display(df)

,date,precipitation
0,2022-06-01 00:00:00,0.002379
1,2022-06-01 01:00:00,0.000000
2,2022-06-01 02:00:00,0.000000
3,2022-06-01 03:00:00,0.000000
4,2022-06-01 04:00:00,0.000000
...,...,...
10267,2022-12-31 19:00:00,0.006940
10268,2022-12-31 20:00:00,0.007048
10269,2022-12-31 21:00:00,0.007385
10270,2022-12-31 22:00:00,0.007488


In [16]:
import xarray as xr

# Load the NetCDF file using xarray
dataset = xr.open_dataset("london_precipitation_2022.nc")

# Inspect the dataset
print(dataset)

<xarray.Dataset>
Dimensions:    (longitude: 1, latitude: 1, expver: 2, time: 5136)
Coordinates:
  * longitude  (longitude) float32 -0.013
  * latitude   (latitude) float32 51.53
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 2022-06-01 ... 2022-12-31T23:00:00
Data variables:
    tp         (time, expver, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-04-06 13:11:45 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...
